In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import time
from finta import TA
from binance.client import Client

In [ ]:
aperturas = []
cierres = []
payload={}
headers = {}
url_btc = "https://api.binance.com/api/v3/klines?symbol=BTCUSDT&interval=3m"

In [ ]:
telegram_ids = ['id']

In [ ]:
##Telegram function and ID´s
def bot_send_text(bot_message):
    for i in telegram_ids:
        bot_token = 'telegram_bot_token'
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + i + '&parse_mode=Markdown&text=' + bot_message
        response = requests.get(send_text)

In [ ]:
def least_squares_ma(data, lookback, close_column):
    
    ma_data = data[:, close_column]
    
    pl = []; lr = [];
    
    for i in range(len(ma_data)):
        
        pl.append(float(ma_data[i]));
        
        if(len(pl) >= lookback):
            
            sum_x = 0.0; sum_y = 0.0; sum_xy = 0.0; sum_xx = 0.0; sum_yy = 0.0;
            
            for a in range(1, len(pl)+1):
                
                sum_x += a;
                sum_y += pl[a-1];
                sum_xy += (pl[a-1] * a);
                sum_xx += (a*a);
                sum_yy += (pl[a-1] * pl[a-1]);
                
            m = ((sum_xy - sum_x * sum_y / lookback) / (sum_xx - sum_x * sum_x / lookback));
            b = sum_y / lookback - m * sum_x / lookback;
            lr.append(m * lookback + b);
            pl = pl[1:];
            
    lr = np.array(lr)  
    
    
    data = data[lookback - 1:, ]  
    
    lr = np.reshape(lr, (-1, 1))
    
    lr = pd.DataFrame(lr,columns=['LSMA'])
    lr.index = df.index[20:]
   
    return lr

In [ ]:
user_key = 'user_key'
secret_key = 'secret_key'
binance_client = Client(user_key, secret_key)

In [ ]:
while True:
    try:
        #Cada un minuto comprueba la existencia de SL en caso de estar abierta la posición:
        if (float(dt.datetime.now().strftime("%M")) % 3 == 0) & (float(dt.datetime.now().strftime("%S")) == 1):

            response_btc = requests.request("GET", url_btc, headers=headers, data=payload)
            btc_spot = json.loads(response_btc.text)
            df = pd.DataFrame(btc_spot)
            df.columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
            df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.open_time]
            acumulado = df
            acumulado = acumulado.astype(float)              
            df = df.drop(columns=['open_time','close_time'])
            df = df.astype(float)
            df['LSMA'] = least_squares_ma(df[['open','high','low','close']].values,21,3)
            df['BB_MIDDLE'] = TA.BBANDS(df[['open','high','low','close']])['BB_MIDDLE']
            df['BB_UPPER'] = TA.BBANDS(df[['open','high','low','close']])['BB_UPPER']
            df['RSI'] = TA.RSI(df[['open','high','low','close']])
            df = df[:-1]
            df = df[-1:]
            df = df.drop(['volume', 'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore'], axis=1)
            
            a = float(binance_client.get_asset_balance(asset = "BTC")["free"])
            b = float(binance_client.get_asset_balance(asset = "BTC")["locked"])            
            
            if ((a == 0) & (b == 0)): 

                #Evualate conditions
                if (
                    (df['LSMA'].values[0] < df['BB_MIDDLE'].values[0]) 
                    & (df['close'].values[0] > df['LSMA'].values[0])
                   ):
                    capital = round(float(binance_client.get_asset_balance(asset = "USDT")['free']) * 0.98,0)

                    ## Sell Orderd 
                    spot_order = binance_client.create_order(
                    symbol='BTCUSDT',
                    type='MARKET',
                    side='BUY',
                    quoteOrderQty=capital, ## Ver cómo calcular.... con el 
                    )

                    aperturas.append(float(spot_order['fills'][0]['price']))
                    
                    sl = 0.999
                    
                    bot_send_text(f"Conitions met. Position is opened at BTC price: {aperturas[-1]}")

                else:
                    None


            elif ((a != 0) | (b != 0)):
                
                
                #SL trigger
                if (df['close'].values[0] <= aperturas[-1]*sl):
                    
                    active_position = binance_client.get_asset_balance(asset = "BTC")["free"]
                    ## Close active position
                    close_position = binance_client.create_order(symbol='BTCUSDT',
                    type='MARKET',
                    side='SELL',
                    quantity=active_position,
                    )

                    cierres.append(float(close_position['fills'][0]['price']))
                    bot_send_text(f"Actual price lower than SL. Result: {round(((cierres[-1]/aperturas[-1])-1)*100,4)}%")
                    
                else:
                    None
 
                if (df['close'].values[0] > df['LSMA'].values[0]):

                        bot_send_text("Actual close price higher than LSMA. Position open.")
                else:
                    active_position = binance_client.get_asset_balance(asset = "BTC")["free"]

                    ## Close active position
                    close_position = binance_client.create_order(symbol='BTCUSDT',
                    type='MARKET',
                    side='SELL',
                    quantity=active_position,
                    )

                    cierres.append(float(close_position['fills'][0]['price']))
                    bot_send_text(f"Actual close price lower than LSMA. Position is closed. Result: {round(((cierres[-1]/aperturas[-1])-1)*100,4)}%")
            

            else:
                None
        else:
            None
    except Exception as e:
        
        print(e)
    time.sleep(1)